In [1]:
import requests 
import pandas as pd
import numpy as np
import math
import os 
from statistics import mean, median, mode
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
MAX_RECORDS = 200
latmin = 43.6406
latmax = 43.736439
lngmax = -79.252357       
lngmin = -79.549301
records = 2000
x = 2000
records_per_page = min(records, MAX_RECORDS)
pages = math.ceil(records / MAX_RECORDS)

maindf = pd.DataFrame({"Address":''},index=[np.arange(0, records)])
maindf["Latitude"]=""
maindf["Longitude"]=""
maindf["Sale_Price"]= ""    
maindf['Mls Number']=""

for page in np.arange(pages):
    url = 'https://api2.realtor.ca/Listing.svc/PropertySearch_Post'
    body = {
        "CultureId":1,
        "ApplicationId":1,
        "PropertySearchTypeId":1,
        "TransactionTypeID":2,
        "BuildingTypeId":0,
        "LongitudeMin":f'{lngmin}',
        "LongitudeMax":f'{lngmax}',
        "LatitudeMin":f'{latmin}',
        "LatitudeMax":f'{latmax}',
        "PriceMin":400000,
        "RecordsPerPage": records_per_page,
        "CurrentPage": page + 1
    }

    response = requests.post(url, data=body).json() 
    
    for index, mls_post in enumerate(response['Results']):
        if page > 0:
            index += (MAX_RECORDS * x)
            
        try:                        
            lng = float(mls_post['Property']['Address']['Longitude'])
            address = mls_post['Property']['Address']
    
            if (lng >= lngmin and lng <= lngmax and address['AddressText'] not in maindf['Address'].unique()):
                maindf.loc[index, "Latitude"] = address['Latitude']  
                maindf.loc[index, "Longitude"] = address['Longitude']
                maindf.loc[index,"Address"] = address['AddressText']        
                maindf.loc[index,"Mls Number"]= mls_post['MlsNumber']
                    
                try:
                    maindf.loc[index,"Sale_Price"] = mls_post['Property']['Price']
                except:
                    maindf.loc[index,"Sale_Price"] = ""     
                
        except Exception as e:
            print('Error ', e)  
            
maindfclean = maindf.loc[maindf['Address']!="",:]

In [3]:
maindfclean[maindfclean['Address'] == ""]
maindfclean['Address'].count()

398

In [31]:
maindfclean = maindf.loc[maindf['Address']!="",:]
maindfclean.set_index('Mls Number')
maindfclean.head()

,Address,Latitude,Longitude,Sale_Price,Mls Number
0,"#2204 -3 MASSEY SQ|Toronto, Ontario M4C5L5",43.6968278,-79.2934124,"$404,900",E4641454
1,"#4811 -355 KING ST W|Toronto, Ontario M5V1J6",43.6460089,-79.3919615,"$405,000",C4634382
2,"#2110 -5 MASSEY SQ|Toronto, Ontario M4C5L6",43.6961831,-79.2929566,"$409,900",E4653105
3,"#1183 -1 GREYSTONE WALK DR|Toronto, Ontario M...",43.7235629,-79.2525218,"$414,900",E4645181
4,"#1403 -238 ALBION RD|Toronto, Ontario M9W6A7",43.7230155,-79.5436818,"$418,000",W4597587


In [37]:
maindfclean['Street']=maindfclean['Address'].str.split('|').str[0]
maindfclean['City - Province']=maindfclean['Address'].str.split('|').str[1]
maindfclean['City Province']=maindfclean['City - Province'].str[:-6]
maindfclean['City']=maindfclean['City Province'].str.split(',').str[0]
maindfclean['Province']=maindfclean['City Province'].str.split(',').str[1]
maindfclean['Postal_Code']=maindfclean['Address'].str[-6:]
maindfclean.head()

C:\Users\pasqu\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\pasqu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\pasqu\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

,Address,Latitude,Longitude,Sale_Price,Mls Number,Street,City - Province,City Province,City,Province,Postal_Code
0,"#2204 -3 MASSEY SQ|Toronto, Ontario M4C5L5",43.6968278,-79.2934124,"$404,900",E4641454,#2204 -3 MASSEY SQ,"Toronto, Ontario M4C5L5","Toronto, Ontario",Toronto,Ontario,M4C5L5
1,"#4811 -355 KING ST W|Toronto, Ontario M5V1J6",43.6460089,-79.3919615,"$405,000",C4634382,#4811 -355 KING ST W,"Toronto, Ontario M5V1J6","Toronto, Ontario",Toronto,Ontario,M5V1J6
2,"#2110 -5 MASSEY SQ|Toronto, Ontario M4C5L6",43.6961831,-79.2929566,"$409,900",E4653105,#2110 -5 MASSEY SQ,"Toronto, Ontario M4C5L6","Toronto, Ontario",Toronto,Ontario,M4C5L6
3,"#1183 -1 GREYSTONE WALK DR|Toronto, Ontario M...",43.7235629,-79.2525218,"$414,900",E4645181,#1183 -1 GREYSTONE WALK DR,"Toronto, Ontario M1K5J3","Toronto, Ontario",Toronto,Ontario,M1K5J3
4,"#1403 -238 ALBION RD|Toronto, Ontario M9W6A7",43.7230155,-79.5436818,"$418,000",W4597587,#1403 -238 ALBION RD,"Toronto, Ontario M9W6A7","Toronto, Ontario",Toronto,Ontario,M9W6A7


In [39]:
df = maindfclean.drop(maindfclean.columns[[0, 1, 2, 3, 6, 7]], axis=1)   
df.head()

,Mls Number,Street,City,Province,Postal_Code
0,E4641454,#2204 -3 MASSEY SQ,Toronto,Ontario,M4C5L5
1,C4634382,#4811 -355 KING ST W,Toronto,Ontario,M5V1J6
2,E4653105,#2110 -5 MASSEY SQ,Toronto,Ontario,M4C5L6
3,E4645181,#1183 -1 GREYSTONE WALK DR,Toronto,Ontario,M1K5J3
4,W4597587,#1403 -238 ALBION RD,Toronto,Ontario,M9W6A7
